In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
IDir = r'/Volumes/PhD/imagery/masters/ROC/M1_ROC.shp'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/'
dNBR_df = gpd.read_file(IDir)
print(dNBR_df.columns)

Index(['BASIN_ID', 'Shape_Leng', 'Shape_Area', 'Segment_ID', 'Acc_Cl',
       'Perim_Buff', 'InsidePeri', 'OutsidePer', 'Fire_ID', 'Fire_Segme',
       'L_X1', 'L_X2', 'L_X3', 'V_X1', 'V_X2', 'Shape_Le_1', 'R', 'V_X3', 'X',
       'ExpX', 'P', 'PCl', 'PCl_Legend', 'LnV', 'Volume', 'VolMin', 'VolMax',
       'VolCl', 'VolCl_Lege', 'CombHaz', 'CombHazCl', 'CombHazCl_', 'area',
       'Field', 'ROC', 'nearest_st', 'distance_t', 'RI_index', 'RI',
       'Mean_dNBR', 'Med_dNBR', 'PF_dNBR', 'geometry'],
      dtype='object')


In [3]:
def M1(dnbr, Value):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['L_X1']
    x_3 = dnbr['L_X3']
    x_2 = dnbr[f'{Value}']
    R = dnbr['RI']
    
    lnx =  b + (b_1 * x_1 * R) + (b_2 * x_2 * R) + (b_3 * x_3 * R)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return prob
    

In [4]:
shp = r'/Volumes/PhD/imagery/masters/baer/thm2017_Segment_DFPredictions_15min_40mmh.shp'
shp = gpd.read_file(shp)
basin_id = 12042
basin_data = shp[shp['Segment_ID'] == basin_id]
print(basin_data['P'])

12424    0.414864
Name: P, dtype: float64


In [6]:
M1P_Mean = M1(dNBR_df, 'Mean_dNBR')
M1P_Median = M1(dNBR_df, 'Med_dNBR')
M1P_PF = M1(dNBR_df, 'PF_dNBR')
M1P_df = {
    'M1P_Mean': M1P_Mean,
    'M1P_Med': M1P_Median,
    'M1p_PF': M1P_PF
}
M1P_df = pd.DataFrame(M1P_df)

merged_gdf = gpd.GeoDataFrame(
    pd.merge(dNBR_df, M1P_df, left_index=True, right_index=True, how="left")
)


In [12]:
merged_gdf

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,distance_t,RI_index,RI,Mean_dNBR,Med_dNBR,PF_dNBR,geometry,M1P_Mean,M1P_Med,M1p_PF
0,17499,7120.0,1.429800e+06,17499,1.0,10,100,0,thm2017,thm2017_17499,...,2055.346175,23,5.08,0.471854,0.477169,0.459052,"POLYGON ((295619.84 3816329.559, 295619.84 381...",0.449618,0.454098,0.438860
1,3115,1120.0,4.180000e+04,3115,1.0,10,100,10,thm2017,thm2017_3115,...,1595.604025,29,7.11,0.072363,0.014718,-0.019005,"MULTIPOLYGON (((298029.84 3825079.559, 298029....",0.148129,0.116710,0.101142
2,3589,4020.0,4.313000e+05,3589,1.0,10,100,0,thm2017,thm2017_3589,...,1881.463927,29,7.11,0.216498,0.051373,-0.020064,"POLYGON ((297059.84 3824619.559, 297069.84 382...",0.301476,0.164258,0.122691
3,5683,4420.0,5.659000e+05,5683,1.0,10,101,12,thm2017,thm2017_5683,...,1702.372795,21,12.95,0.692429,0.702799,0.724929,"POLYGON ((283709.84 3822919.559, 283689.84 382...",0.999180,0.999251,0.999382
4,5920,16620.0,7.486600e+06,5920,1.0,10,100,0,thm2017,thm2017_5920,...,1525.053372,37,6.10,0.605956,0.651802,0.753237,"POLYGON ((303389.84 3819559.559, 303389.84 381...",0.848432,0.870989,0.910869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,15065,1720.0,9.550000e+04,15065,1.0,10,100,0,thm2017,thm2017_15065,...,1173.707451,10,9.14,0.164376,0.111513,0.055359,"POLYGON ((289119.84 3817119.559, 289079.84 381...",0.313190,0.248063,0.189564
733,35852,2080.0,8.360000e+04,35852,1.0,10,101,12,thm2017,thm2017_35852,...,2151.287445,25,7.37,0.284851,0.279801,0.261897,"MULTIPOLYGON (((278059.84 3804989.559, 278069....",0.473049,0.466838,0.444912
734,36553,1460.0,4.570000e+04,36553,1.0,10,100,11,thm2017,thm2017_36553,...,1675.063365,25,7.37,0.348485,0.339397,0.331989,"MULTIPOLYGON (((277049.84 3804469.559, 277049....",0.468316,0.457159,0.448097
735,36955,2380.0,1.098000e+05,36955,1.0,10,100,11,thm2017,thm2017_36955,...,1352.291607,25,7.37,0.208930,0.202108,0.175856,"POLYGON ((277699.84 3804059.559, 277689.84 380...",0.281668,0.274903,0.249830


In [11]:
merged_gdf.to_file(r'/Volumes/PhD/imagery/masters/ROC/M1_CT_ROC.shp')